In [38]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
import pandas as pd
import numpy as np
from data_prep import *

In [40]:
df = fetch_data(num_currencies=2)
df = interpolate(df)
df = set_target(df, 1)
df.head()

,EURO,POUND,target
day,,,
2000-01-03,0.9847,0.6146,0
2000-01-04,0.9700,0.6109,0
2000-01-05,0.9676,0.6092,1
2000-01-06,0.9686,0.6070,1
2000-01-07,0.9714,0.6104,1


In [58]:
targets = df['target']
features = df.drop('target', axis=1)

## Train test Split

In [59]:
def train_test_split(features, targets, percentile=None, test_window=None):
    """
    :param percentile: float or None
    :param test_window: tuple or None
    """
    if percentile is not None:
        cutoff = int(features.shape[0]*percentile)
        X_train, X_test = features[:cutoff], features[cutoff:]
        y_train, y_test = targets[:cutoff], targets[cutoff:]
    elif test_window is not None:
        start = test_window[0]
        end = test_window[1]
        X_train, X_test = features[:start], features[start:end]
        y_train, y_test = targets[:start], cutoff[start:end]
    return X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(features, targets, percentile=0.7)

## Scale data

In [53]:
from sklearn.preprocessing import MinMaxScaler

In [60]:
scaler = MinMaxScaler()
scaled_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)
scaled_test = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)

## Reshape Data

In [62]:
def backstep_columns(df, columns=['EURO'], steps=3):
    N = df.shape[0] - steps
    # Increment steps by 1 to account for the current day
    steps = steps +1
    output = np.zeros((N, steps, len(columns)))
    for i, c in enumerate(columns):
        for j in range(steps):
            # Fill in backwards, so the oldest is last
            output[:,steps-j-1,i] = df[c].shift(j)[steps-1:]
    return output
shaped_train = backstep_columns(scaled_train, ['EURO', 'POUND'], 5)
shaped_test = backstep_columns(scaled_test, ['EURO', 'POUND'], 5)
        
print(shaped_test.shape)
shaped_test[:3]

(2186, 6, 2)


array([[[0.17293876, 0.50154799],
        [0.17789942, 0.51083591],
        [0.18286008, 0.52012384],
        [0.18884707, 0.52283282],
        [0.18793477, 0.52334881],
        [0.18702247, 0.52386481]],

       [[0.17789942, 0.51083591],
        [0.18286008, 0.52012384],
        [0.18884707, 0.52283282],
        [0.18793477, 0.52334881],
        [0.18702247, 0.52386481],
        [0.18611016, 0.5243808 ]],

       [[0.18286008, 0.52012384],
        [0.18884707, 0.52283282],
        [0.18793477, 0.52334881],
        [0.18702247, 0.52386481],
        [0.18611016, 0.5243808 ],
        [0.18764967, 0.52321981]]])

## Convert to Pytorch Data

In [70]:
from model_utils import BasicData
from torch.utils.data import DataLoader
from torch import nn

In [71]:
training_data = BasicData(shaped_train, y_train)
test_data = BasicData(shaped_test, y_test)
train_loader = DataLoader(training_data, batch_size=64, shuffle=True, drop_last=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=True, drop_last=True)

In [ ]:
class LSTM(nn.Module):

    def __init__(self, input_size, hidden_size, num_layers, out_d):
        super(LSTM, self).__init__()

        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_length = seq_length

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True)

        self.fc = nn.Linear(hidden_size, out_d)

    def forward(self, x):
        h_0 = torch.zeros(
            self.num_layers, x.shape[0], self.hidden_size)

        c_0 = torch.zeros(
            self.num_layers, x.shape[0], self.hidden_size)

        # Propagate input through LSTM
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))
        print(h_out.shape)

        h_out = h_out.view(-1, self.hidden_size)
        print(h_out.shape)

        out = self.fc(h_out)

        return out